In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-10-02"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
2123,2024-10-02,Europa Supercopa Feminina,13:00,Fenerbahce F,Besiktas F,1.01,12.20,155.5,1.87,1.79,-20.5,2.00,1.72,IclLuba0,0.990099,0.081967,0.534759,0.558659,0.072066,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0,0,0.00,0.00,0.000,0.000,1.197960,0.030912,0.106446,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
2124,2024-10-02,Itália Série A2,15:30,Assigeco Piacenza,Rimini,2.41,1.47,156.5,1.85,1.81,3.5,1.95,1.72,AFJFlnoT,0.414938,0.680272,0.540541,0.552486,0.095210,238.524,68.597291,0.287591,1.2,1.643168,1.369306,205.36,3.336,1.259714,0.377612,-39.0,139.590,31.526893,0.225854,1.2,1.643168,1.369306,164.05,1.866,0.458454,0.245688,20.0,68,85,3.02,1.93,172.006,214.244,0.342619,0.015456,0.088629,2.30,0.460,3.065217,0.657755,0.7,0.042245,4.47,0.894,0.525727,0.372857,0.6,0.227143
2125,2024-10-02,Itália Série A2,15:30,Forli,Cento,1.21,3.74,151.5,1.84,1.82,-10.5,1.95,1.72,UXVEJ6p3,0.826446,0.267380,0.543478,0.549451,0.093826,126.540,32.253595,0.254889,0.6,1.341641,2.236068,111.78,1.712,0.513877,0.300162,-23.0,121.286,36.366607,0.299842,1.8,1.643168,0.912871,184.14,1.690,0.719757,0.425892,-15.0,69,62,1.62,2.97,102.500,183.576,0.722820,0.007728,0.088629,-1.47,-0.294,-0.714286,0.783763,0.7,-0.083763,-0.62,-0.124,-22.096774,0.423191,0.5,0.076809
2126,2024-10-02,Itália Série A2,15:30,JuVi Cremona,Cividale,2.57,1.42,152.5,1.88,1.78,4.5,1.93,1.74,2RwTe3hd,0.389105,0.704225,0.531915,0.561798,0.093330,249.484,108.923444,0.436595,0.6,1.341641,2.236068,178.54,3.460,1.004764,0.290394,-71.0,196.196,42.361673,0.215915,1.2,1.643168,1.369306,161.70,2.704,0.661536,0.244651,-19.0,79,77,2.26,2.10,113.666,216.642,0.407605,0.038640,0.073215,-3.57,-0.714,-2.198880,0.580496,0.4,-0.180496,1.70,0.340,1.235294,0.384306,0.6,0.215694
2127,2024-10-02,Itália Série A2,15:30,Pesaro,Vigevano,1.15,4.49,159.5,1.88,1.78,-12.5,1.97,1.71,zwseaxal,0.869565,0.222717,0.531915,0.561798,0.092282,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,246.470,106.160347,0.430723,0.6,1.341641,2.236068,112.42,3.468,1.564599,0.451153,-54.0,0,73,0.00,1.54,0.000,276.060,0.837495,0.038640,0.099917,0.00,0.000,inf,0.000000,0.0,0.000000,-5.00,-1.000,-3.490000,0.255468,0.3,0.044532
2128,2024-10-02,Itália Série A2,15:30,Udine,Nardo,1.12,4.97,151.5,1.84,1.82,-13.5,2.01,1.68,MgmY6T0c,0.892857,0.201207,0.543478,0.549451,0.094064,138.398,30.301447,0.218944,1.2,1.643168,1.369306,123.54,1.866,0.499780,0.267835,10.0,146.732,49.019914,0.334078,1.8,1.643168,0.912871,140.25,1.906,0.652940,0.342571,4.0,71,85,1.74,1.65,120.328,0.000,0.894043,0.007728,0.126474,-0.93,-0.186,-0.645161,0.756976,0.7,-0.056976,0.00,0.000,inf,0.000000,0.0,0.000000
2129,2024-10-02,Itália Série A2,15:30,Verona,Urania Milano,1.56,2.21,160.5,1.83,1.83,-4.5,1.95,1.72,ATbl3o0M,0.641026,0.452489,0.546448,0.546448,0.093514,192.946,57.527710,0.298154,1.8,1.643168,0.912871,187.20,2.442,0.920473,0.376934,-6.0,184.230,51.517955,0.279639,1.2,1.643168,1.369306,163.56,2.358,0.726684,0.308178,-2.0,96,94,1.95,1.74,131.380,250.874,0.243830,0.000000,0.088629,-1.16,-0.232,-2.413793,0.706407,0.7,-0.006407,0.65,0.130,9.307692,0.368099,0.4,0.031901
2130,2024-10-02,Mundo Amistoso 

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

Sem jogos
